<a href="https://colab.research.google.com/github/Mageed-Ghaleb/OptimizationSystems-Course/blob/main/Lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4 — Two-Stage Stochastic Optimization (Pyomo) + Robust Stress Test

**Theme:** uncertainty, scenario sampling, and robustness checks.

## What you will do
1. Build and solve a **two-stage stochastic** capacity / production-planning model using **scenario sampling** (SAA).
2. **Stress-test** the stochastic solution against a **robust uncertainty set** (box/interval demand).
3. Solve a simple **robust** decision model and compare trade-offs.

> This notebook is self-contained for Google Colab.


## Learning objectives
By the end, you should be able to:
- Write a two-stage stochastic program in **extensive form** (scenarios) in Pyomo.
- Use **scenario sampling** to approximate an expected-value objective.
- Evaluate a policy under **worst-case** demand within a specified uncertainty set.
- Compare stochastic vs robust first-stage decisions.


In [ ]:
# ===== Colab setup (run once) =====
# Pyomo (modeling) + solvers (CBC/GLPK)

!pip -q install pyomo pandas numpy matplotlib

# Install open-source MILP/LP solvers.
# CBC is usually fast; GLPK is widely available.
!apt-get -qq update
!apt-get -qq install -y coinor-cbc glpk-utils > /dev/null

import shutil
print('cbc  found:', shutil.which('cbc'))
print('glpsol found:', shutil.which('glpsol'))


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyomo.environ import (
    ConcreteModel, Var, NonNegativeReals, Objective, Constraint, Param, Set, value, SolverFactory, minimize
)

np.set_printoptions(precision=4, suppress=True)


## Problem: one-period capacity + recourse (newsvendor-style)

You choose a **first-stage** capacity/production level \(x\) *before* demand is known.
After demand \(d_\omega\) is realized (scenario \(\omega\)), you choose **second-stage** recourse:
- \(s_\omega\) = unmet demand (shortage)  \(\ge 0\)
- \(u_\omega\) = leftover inventory (overage) \(\ge 0\)

**Balance (per scenario):**
\[
x + s_\omega - u_\omega = d_\omega
\]

**Costs:**
- First-stage cost: \(c x\)
- Shortage penalty: \(p\, s_\omega\)
- Holding/overage cost: \(h\, u_\omega\)

**Two-stage stochastic objective (expected cost):**
\[
\min_{x \ge 0}\; c x + \mathbb{E}[p s + h u]
\]

We approximate the expectation using a sampled scenario set (SAA).


## Step 1 — Generate scenarios by sampling (SAA)
We will assume demand is random and generate \(N\) scenarios.

You can swap in any distribution that fits your course context.


In [ ]:
# ---- Demand distribution and sampling ----
seed = 7
rng = np.random.default_rng(seed)

N = 50  # number of scenarios (try 20, 50, 200)
mu = 100.0
sigma = 20.0

# Truncated normal (lower bound 0)
demand = np.maximum(0.0, rng.normal(loc=mu, scale=sigma, size=N))

# Empirical probabilities (uniform)
prob = np.ones(N) / N

print('Sample demand summary:')
print(pd.Series(demand).describe())

plt.figure()
plt.hist(demand, bins=12)
plt.xlabel('Demand')
plt.ylabel('Count')
plt.title('Sampled demand scenarios')
plt.show()


## Step 2 — Build the extensive-form Pyomo model (two-stage)

Model variables:
- \(x\) : first-stage decision
- \(s_\omega, u_\omega\) : second-stage recourse for each scenario

Solve as one LP (because everything is linear).


In [ ]:
# ---- Build extensive form in Pyomo ----
m = ConcreteModel()

m.O = Set(initialize=range(N))

m.d = Param(m.O, initialize={i: float(demand[i]) for i in range(N)})
m.pw = Param(m.O, initialize={i: float(prob[i]) for i in range(N)})

# Costs
c = 4.0   # unit capacity/production cost
p = 12.0  # shortage penalty
h = 1.5   # holding cost

m.x = Var(domain=NonNegativeReals)
m.s = Var(m.O, domain=NonNegativeReals)
m.u = Var(m.O, domain=NonNegativeReals)

def balance_rule(m, i):
    return m.x + m.s[i] - m.u[i] == m.d[i]

m.balance = Constraint(m.O, rule=balance_rule)

def obj_rule(m):
    return c*m.x + sum(m.pw[i]*(p*m.s[i] + h*m.u[i]) for i in m.O)

m.obj = Objective(rule=obj_rule, sense=minimize)

# ---- Solve ----
solver = SolverFactory('cbc') if SolverFactory('cbc').available() else SolverFactory('glpk')
res = solver.solve(m, tee=False)

x_star = value(m.x)
exp_cost = value(m.obj)

print('Solver:', solver)
print('x* (stochastic) =', x_star)
print('Estimated expected cost (SAA) =', exp_cost)

# Show a few scenario recourse decisions
sample_rows = []
for i in range(min(10, N)):
    sample_rows.append({
        'scenario': i,
        'demand': value(m.d[i]),
        'shortage s': value(m.s[i]),
        'overage u': value(m.u[i]),
        'recourse cost': p*value(m.s[i]) + h*value(m.u[i])
    })

pd.DataFrame(sample_rows)


## Step 3 — Stress-test the stochastic solution against a robust demand set

Define a **box/interval uncertainty set** for demand:
\[
\mathcal{U} = [d_{\min}, d_{\max}]
\]

For a *fixed* first-stage decision \(x\), the realized total cost at demand \(d\) (with optimal recourse) is:
\[
	ext{Cost}(x; d) = c x + p\,\max(d-x, 0) + h\,\max(x-d, 0)
\]

The **worst-case cost** over \(\mathcal{U}\) is:
\[
	ext{WC}(x) = \max_{d\in[d_{\min}, d_{\max}]} 	ext{Cost}(x; d)
\]

Because \(	ext{Cost}(x; d)\) is piecewise-linear in \(d\), the maximum over an interval occurs at an endpoint.
So:
\[
	ext{WC}(x) = \max\{	ext{Cost}(x; d_{\min}),\ 	ext{Cost}(x; d_{\max})\}
\]


In [ ]:
def realized_cost(x, d, c=4.0, p=12.0, h=1.5):
    # optimal recourse: shortage=max(d-x,0), overage=max(x-d,0)
    return c*x + p*max(d-x, 0.0) + h*max(x-d, 0.0)

# Choose robust interval from quantiles (you can replace with a designed engineering set)
d_min = float(np.quantile(demand, 0.05))
d_max = float(np.quantile(demand, 0.95))

wc_xstar = max(realized_cost(x_star, d_min, c,p,h), realized_cost(x_star, d_max, c,p,h))
argmax = d_min if realized_cost(x_star, d_min, c,p,h) >= realized_cost(x_star, d_max, c,p,h) else d_max

print(f'Robust interval U = [{d_min:.2f}, {d_max:.2f}] (from 5% and 95% sample quantiles)')
print(f'Worst-case cost of stochastic x* over U: {wc_xstar:.2f} (attained at d={argmax:.2f})')

# Compare to expected cost estimate
print(f'SAA expected cost at x*: {exp_cost:.2f}')


## Step 4 — Solve the robust decision model on the same interval

Robust model (minimize worst-case cost):
\[
\min_{x\ge 0}\ \max\{	ext{Cost}(x; d_{\min}),\ 	ext{Cost}(x; d_{\max})\}
\]

We can solve this as an LP by introducing a variable \(z\) for the worst-case cost:
\[
\min_{x,z}\ z\quad 	ext{s.t.}\quad z \ge 	ext{Cost}(x; d_{\min}),\ \ z \ge 	ext{Cost}(x; d_{\max}),\ x\ge 0
\]

This is a standard trick: a minimax of finitely many convex functions becomes a linear (here: piecewise-linear) program.


In [ ]:
# ---- Robust minimax model over two endpoints ----
mr = ConcreteModel()
mr.x = Var(domain=NonNegativeReals)
mr.z = Var(domain=NonNegativeReals)

# auxiliary nonnegative vars to represent max() terms at each endpoint
mr.s_lo = Var(domain=NonNegativeReals)
mr.u_lo = Var(domain=NonNegativeReals)
mr.s_hi = Var(domain=NonNegativeReals)
mr.u_hi = Var(domain=NonNegativeReals)

# balance constraints for each endpoint demand
mr.b_lo = Constraint(expr= mr.x + mr.s_lo - mr.u_lo == d_min)
mr.b_hi = Constraint(expr= mr.x + mr.s_hi - mr.u_hi == d_max)

# z >= cost at each endpoint
mr.z_lo = Constraint(expr= mr.z >= c*mr.x + p*mr.s_lo + h*mr.u_lo)
mr.z_hi = Constraint(expr= mr.z >= c*mr.x + p*mr.s_hi + h*mr.u_hi)

mr.obj = Objective(expr=mr.z, sense=minimize)

solver = SolverFactory('cbc') if SolverFactory('cbc').available() else SolverFactory('glpk')
res = solver.solve(mr, tee=False)

x_rob = value(mr.x)
wc_rob = value(mr.z)

print('x_R (robust minimax) =', x_rob)
print('Worst-case cost at optimum =', wc_rob)

# Compare the two decisions under both metrics
wc_stoch = wc_xstar

# expected cost of robust decision under the same sampled scenarios
exp_cost_rob = c*x_rob + np.mean([p*max(d-x_rob,0.0) + h*max(x_rob-d,0.0) for d in demand])

print('
Comparison:')
print(f'Stoch x*:  x={x_star:.2f},  SAA-ExpCost={exp_cost:.2f},  WC(U)={wc_stoch:.2f}')
print(f'Robust xR: x={x_rob:.2f},  SAA-ExpCost={exp_cost_rob:.2f},  WC(U)={wc_rob:.2f}')


## Visualization — trade-off curves
We will scan \(x\) values and plot:
- SAA expected cost estimate
- worst-case cost over \([d_{\min}, d_{\max}]\)


In [ ]:
xs = np.linspace(0, max(d_max, x_rob, x_star)*1.3, 200)
exp_vals = []
wc_vals = []
for x in xs:
    exp_vals.append(c*x + np.mean([p*max(d-x,0.0) + h*max(x-d,0.0) for d in demand]))
    wc_vals.append(max(realized_cost(x, d_min, c,p,h), realized_cost(x, d_max, c,p,h)))

plt.figure()
plt.plot(xs, exp_vals, label='SAA expected cost')
plt.axvline(x_star, linestyle='--', label='x* (stochastic)')
plt.axvline(x_rob, linestyle='--', label='x_R (robust)')
plt.xlabel('x')
plt.ylabel('Cost')
plt.title('Expected-cost objective')
plt.legend()
plt.show()

plt.figure()
plt.plot(xs, wc_vals, label='Worst-case cost over U')
plt.axvline(x_star, linestyle='--', label='x* (stochastic)')
plt.axvline(x_rob, linestyle='--', label='x_R (robust)')
plt.xlabel('x')
plt.ylabel('Cost')
plt.title('Robust worst-case objective')
plt.legend()
plt.show()


## Optional extension — “budgeted uncertainty” with two demand components

If demand has two components \((d_1, d_2)\) (e.g., two products or two regions), a **budgeted** uncertainty set can limit how many components are simultaneously at their worst-case.

A simple 2D budgeted set example:
\[
 d_i = ar d_i + \Delta_i z_i,\quad 0\le z_i \le 1,\quad z_1 + z_2 \le \Gamma
\]
- \(\Gamma=0\): no deviation (nominal)
- \(\Gamma=1\): at most one component fully deviates
- \(\Gamma=2\): both can fully deviate (box worst-case)

Below, we show how to **stress-test** a first-stage decision against all extreme points of this set.
(For \(2\) dimensions, enumeration is easy. For higher dimensions, you typically need optimization-based reformulations.)


In [ ]:
# ---- Budgeted uncertainty stress test demo (2 components) ----

# Suppose a single shared capacity x supplies two independent demands d1, d2.
# Any shortfall incurs penalty p per unit, holding cost h on leftover.
# Here we stress-test a given x against a small budgeted set.

mu1, mu2 = 60.0, 50.0
Delta1, Delta2 = 20.0, 25.0
Gamma = 1.0  # try 0, 1, 2

# Extreme points for 2D budgeted set are limited; we can enumerate candidates.
# z in {0,1} with z1+z2<=Gamma plus possibly fractional when Gamma not integer.
# For teaching, we enumerate a small representative set.

candidates = []
# integer extremes
for z1 in [0.0, 1.0]:
    for z2 in [0.0, 1.0]:
        if z1 + z2 <= Gamma + 1e-9:
            candidates.append((z1,z2))
# fractional extreme when Gamma in (0,2): (Gamma,0) and (0,Gamma)
if 0.0 < Gamma < 2.0:
    candidates.append((min(Gamma,1.0), 0.0))
    candidates.append((0.0, min(Gamma,1.0)))

# unique
candidates = list({(round(a,6),round(b,6)) for a,b in candidates})

print('Budgeted candidates z:', candidates)

x_test = x_star  # stress-test the stochastic solution

worst = -1
worst_point = None
for z1,z2 in candidates:
    d1 = mu1 + Delta1*z1
    d2 = mu2 + Delta2*z2
    d_tot = d1 + d2
    cost = realized_cost(x_test, d_tot, c,p,h)
    if cost > worst:
        worst = cost
        worst_point = (d1,d2)

print(f'Stress test for x={x_test:.2f} under budgeted uncertainty (Gamma={Gamma}):')
print(f'Worst-case cost = {worst:.2f} at (d1,d2)={worst_point}')


## What to submit
1. Your code and results for the two-stage stochastic model (x*, expected cost).
2. Your robust interval \([d_{\min}, d_{\max}]\) choice and the worst-case cost of x*.
3. The robust decision x_R and the comparison table.
4. One short paragraph: when would you prefer stochastic vs robust in this example?
